In [1]:
# setup
from IPython.core.display import display,HTML
display(HTML('<style>.prompt{width: 0px; min-width: 0px; visibility: collapse}</style>'))
display(HTML(open('rise.css').read()))

# imports
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set(style="whitegrid", font_scale=1.5, rc={'figure.figsize':(12, 6)})


# CMPS 2200
# Introduction to Algorithms

## Quicksort


Today's agenda:

- Quicksort

We saw how the problem of selection could be solved with a randomized algorithm. The key was to choose a random element and then partition the list into two parts. 

What if we recursively sorted these two parts?

Let $a=\langle 2, 5, 4, 1, 3, -1, 99\rangle.$ Suppose we chose 4 as the pivot. Then the two parts of the list are $\ell=\langle 2, 1, 3, -1\rangle$ and $r=\langle 5, 99\rangle$. In sorted order they are $\ell'=\langle -1, 1, 2, 3\rangle$ and $r'=\langle 5, 99\rangle$.

So all we have to do is append $l'$, the pivot and $r'$!

This suggests a divide-and-conquer algorithm, but with similar characteristics as our algorithm for selection. 

This sorting algorithm is actually called *Quicksort* (invented in 1959 by C. A. R. Hoare). 

$$\begin{array}{ll}  
\mathit{quicksort}~a =  \\  
~~~~\texttt{if}~|a| = 0~\texttt{then}~a  \\  
~~~~\texttt{else}   \\  
~~~~~~~~\texttt{let}  \\  
~~~~~~~~~~~~p = \texttt{pick a random pivot from}~a  \\  
~~~~~~~~~~~~    a_1 = \left\langle\, x \in a \;|\; x < p \,\right\rangle  \\  
~~~~~~~~~~~~    a_2 = \left\langle\, x \in a \;|\; x = p \,\right\rangle  \\  
~~~~~~~~~~~~    a_3 = \left\langle\, x \in a \;|\; x > p \,\right\rangle  \\  
~~~~~~~~~~~~    (s_1,s_3) = (\mathit{quicksort}~a_1)~\mid\mid{}~(\mathit{quicksort}~a_3)  \\  
~~~~~~~~   \texttt{in}  \\  
~~~~~~~~~~~~    s_1 \texttt{++}{} a_2 \texttt{++}{} s_3  \\  
~~~~~~~~  \texttt{end}  
\end{array}
$$


In terms of parallelism, we can partition in parallel as before and sort the two parts of the list in parallel.

## Analysis

How should we analyze the work of quicksort?

We'll take a slightly different approach than for selection to estimate the expected work. To account for work, we'll look at the total amount of expected work. 

First, an assumption to streamline our analysis:

> We will "simulate" randomess using priorities: before the start of the algorithm, we assign each key a priority uniformly at random from the real interval $[0, 1]$ such that each key has a unique priority. The algorithm then picks the pivot by selecting the key with the highest priority.

Let the random variable $Y(n)$ be the number of comparisons made by Quicksort on an input of size $n$. Note that the work is $O(Y(n))$, since there is $O(1)$ work done by non-comparisons (i.e., choosing pivots, concatenation of lists).

What we want to know is ultimately how many comparisons are made by quick sort.

Let $X_{ij}$ be an indicator random variable that is 1 if $t_i$ and $t_j$ are ever compared and 0 otherwise. So we have that $Y(n) = \sum_i \sum_j X_{ij}$ and so this means that:

$$ \mathbf{E}\left[Y(n)\right] = \mathbf{E}\left[\sum_i \sum_j X_{ij}\right] = \sum_i \sum_j \mathbf{E}\left[X_{ij}\right] $$

We saw that $\mathbf{E}[X_{ij}] = \mathbf{P}[X_{ij} = 1]$. So when are a pair of elements $t_i$ and $t_j$ compared?

It's useful to consider the sorted version of $a$, indexed as $t_0, t_1, \ldots, t_{n-1}$. 



<img src="qsort.png" alt="quick sort" width="50%"/>

e.g.


$
a = [1,~~~~\mathbf{2},~~~~5,~~~~6,~~~~9,~~~\mathbf{10},~~~~12] \\
p = [0.3, ~0.2, ~0.7, ~0.8, ~0.4, ~0.1, ~0.5]
$


<br>

If any element *between* $t_i$ and $t_j$ in the sorted order is chosen as a pivot *before* $t_i$ or $t_j$, then they will never be compared. 

In Quicksort, two elements $t_i$ and $t_j$ are only ever compared if one of them is a pivot **and** if it has the highest priority in the sequence of elements $t_i$ to $t_j$.

Since the pivot is chosen randomly, the probability that either $t_i$ or $t_j$ has the highest probability in the range of elements from $t_i$ to $t_j$ is:

$$ \mathbf{P}[X_{ij}] = \frac{2}{j - i + 1}. $$

$j - i +1$ is the number of elements between $t_i$ and $t_j$ inlcuding $t_i$ and $t_j$.

<br>

This gives us

$$\begin{align}
\mathbf{E}\left[{Y(n)}\right] &\leq \sum_{i=0}^{n-1} \sum_{j=i+1}^{n} \mathbf{E}\left[{X_{ij}}\right] \\
&=  \sum_{i=0}^{n-1}  \sum_{j=i+1}^{n} \frac{2}{j-i+1} \\
&=  \sum_{i=0}^{n-1}  \sum_{k=1}^{n-i} \frac{2}{k+1} \\
&=  \sum_{i=0}^{n-1}  \sum_{k=1}^{n} \frac{2}{k} \\
&\leq  2 \sum_{i=0}^{n-1}  \ln n      ~~~~~~~~~ \hbox{Harmonic series:} \sum_{k=1}^n \frac{1}{k} < \ln n \\ 
&=  O(n\lg n)
\end{align}$$

Analyzing the span of Quicksort can be done in the same way as we did for selection. That is, if we have a guarantee that at level $d$ of recursion that the larger of the two lists is $(3/4)^d n$, then we can show that the span at each level is $O(\log n)$ (expected). Using the same approach as for selection we can show that the total span is $O(\log^2 n)$ with high probability.

## The Monty Hall Problem

<img width="60%" src="Monty_Hall.png"/>

Suppose you're on a game show, and you're given the choice of three doors: Behind one door is a car; behind the others, goats. You pick a door, say No. 1, and the host, who knows what's behind the doors, opens another door, say No. 3, which has a goat. He then says to you, "Do you want to pick door No. 2?" 

Is it to your advantage to switch your choice?

Why or why not?

This [puzzle](https://en.wikipedia.org/wiki/Monty_Hall_problem) is based on the game show *Let's Make a Deal* and named after the host Monty Hall.


<img width="60%" src="Monty_Hall_standard_probs.png"/>

So the probability of winning by switching is 2/3, while the probability of keeping yoour choice and winning is 1/3. 

<img width="60%" src="Monty_closed_door.png"/>

Suppose you pick Door 1. With the doors closed, there is a 1/3 probability of winning the car.

<img width="60%" src="Monty_open_door.png"/>

With one door open, the odds of that door being winning go to 0 and the odds of Door 2 being winning go to 2/3.

In [her column titled "Ask Marilyn"](https://en.wikipedia.org/wiki/Marilyn_vos_Savant#The_Monty_Hall_problem),  [Marilyn vos Savant](https://en.wikipedia.org/wiki/Marilyn_vos_Savant) suggested that it's useful to think of having 1,000,000 doors. You make a choice, and the host opens 999,998 doors. Should you switch?

<img width="60%" src="Monty_Little_green_alien.png"/>

In this arrangement, we flip a coin after the goat is revealed and have even odds of winning. One way to think about this is that we're just ignoring the information that is given.